In [41]:
import io
import json

import geopandas
import pandas as pd

from statistics import mean

from ipywidgets import HTML

from ipyleaflet import Map, MarkerCluster, Marker, Popup, Icon

# Create GeoDataFrame from geojson features
# json = requests.get("...").json()
with open("../examples/sample.geojson") as fh:
    json = json.load(fh)

gdf = geopandas.GeoDataFrame.from_features(json)

# gdf.dtypes
#geometry    geometry
#date          object
#username      object
#geosong       object
#dtype: object
# Beware of the line below:
# "TypeError: Object of type Timestamp is not JSON serializable"
# gdf["date"] = pd.to_datetime(gdf["date"])

# Create map centered on mean latitude/longitude 
m = Map(zoom=5, center=(mean(gdf.geometry.y),
                        mean(gdf.geometry.x)))

def create_popup(title, artist, username, date):
    html="""
    <div>
    <p><img src="icons8-musical-100.png" width="75" height="75"></p>
    <p><b>{0}</b><br/>{1}<br/><i><a href="https://twitter.com/{2}">@{2}</a></i></p>
    </div>"""
    return HTML(value=html.format(title, artist, username.replace("@",""), date))

# Create markers from GeoDataFrame
# NOTE: GeoData layer does not support custom popups 
#       see https://github.com/jupyter-widgets/ipyleaflet/issues/837
markers = []
for index, row in gdf.iterrows():
    marker = Marker(location=(row.geometry.y, row.geometry.x), 
                    min_width=300)
    marker.popup = create_popup(row.geosong["title"], 
                                row.geosong["artist"],
                                row.username,
                                row.date)
    markers.append(marker)

m.add_layer(MarkerCluster(markers=markers))

display(m)

Map(center=[41.02621495450895, -0.39550781250000006], controls=(ZoomControl(options=['position', 'zoom_in_text…